In [19]:
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit.quantum_info import Operator
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from framework import Framework
from tqdm import tqdm

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

seed = 23094236
np.random.seed(seed=seed)
rng=np.random.default_rng(seed=seed)

In [20]:
### perm ansatz
# [1] Nicola Mariella, Andrea Simonetto, *A Quantum Algorithm for the Sub-Graph Isomorphism Problem*, https://arxiv.org/abs/2111.09732#
# [2] https://github.com/qiskit-community/subgraph-isomorphism

from typing import Union, Tuple
from qiskit.circuit import ParameterVector

IntOrTuple = Union[int, Tuple]

def _hcph(phi: float, qc: QuantumCircuit, qubits: IntOrTuple):
    ctrl, target = qubits if isinstance(qubits, tuple) else (-1, qubits)
    qc.h(target)
    if ctrl >= 0:
        qc.cp(phi, ctrl, target)
    else:
        qc.p(phi, target)
    qc.h(target)


S4_BLOCK_PARCOUNT = 5


def _s4_block(params: np.ndarray) -> QuantumCircuit:
    """Build the basic block S4 for the permutations Ansatz.

    Args:
        params (np.ndarray): The array of parameters.
    """
    params = np.asarray(params).flatten()
    assert params.shape == (S4_BLOCK_PARCOUNT,)
    qc = QuantumCircuit(QuantumRegister(name="q", size=2))
    _hcph(params[0], qc, 0)

    qc.h(1)
    qc.p(params[1], 1)
    qc.cp(params[2], 0, 1)
    qc.h(1)

    _hcph(params[3], qc, (1, 0))
    _hcph(params[4], qc, (0, 1))
    return qc


def _map_qreg(array, qreg: QuantumRegister) -> np.ndarray:
    assert isinstance(qreg, QuantumRegister)
    array = np.asarray(array)
    fun = np.vectorize(lambda i: qreg[i])
    return fun(array)


def _expand_topology(topology, *, qreg: QuantumRegister) -> np.ndarray:
    if isinstance(topology, str):
        if topology in {"linear", "circular"}:
            # v = np.arange(qreg.size - 1)
            # v = np.stack([v, v + 1]).T
            v = np.array([[qreg.size - 1, 0]])
            for i in range(qreg.size):
                for j in range(qreg.size):
                    if i < j:
                        v = np.concatenate([v, np.array([[i, j]])], axis=0)
            # if topology == "circular" and qreg.size > 2:
            #     v = np.concatenate([v, np.array([[qreg.size - 1, 0]])], axis=0)
            topology = v
        else:
            raise ValueError(f"Unrecognized topology: {topology}")
    topology = np.asarray(topology)
    assert topology.ndim == 2 and topology.shape[1] == 2
    return _map_qreg(topology, qreg=qreg)

def params_tensor(shape, *, name="t") -> np.ndarray:
    """Prepare a tensor of circuit parameters."""
    shape = tuple(np.atleast_1d(shape).flatten())
    v = ParameterVector(name=name, length=np.product(shape))
    v = np.array(v.params)
    return v.reshape(shape)

def s4_ansatz(
    topology, *, qreg: Union[QuantumRegister, int], params=None
) -> Tuple[QuantumCircuit, np.ndarray]:
    """Construct the permutations ansatz based on the S4 block.

    Args:
        topology (str, np.ndarray): The topology for the ansatz, see the function
        ansatz() for more information.
        qreg (QuantumRegister, int): The destination quantum register.
        params (np.ndarray): The array of parameters.
    """
    if isinstance(qreg, int):
        qreg = QuantumRegister(qreg)
    assert isinstance(qreg, QuantumRegister)
    topology = _expand_topology(topology, qreg=qreg)
    if params is None:
        params = params_tensor((len(topology), S4_BLOCK_PARCOUNT))
    params = np.asarray(params)
    assert params.ndim == 2 and params.shape[1] == S4_BLOCK_PARCOUNT
    assert len(params) == len(topology)
    qc = QuantumCircuit(qreg)
    for v, q in zip(params, topology):
        qc.compose(_s4_block(v), qubits=q, inplace=True)

    qc_ = QuantumCircuit(qreg)
    qc_.compose(qc.to_gate(label="PermAnsatz"), inplace=True)
    return qc_, params

def thetas_to_prob(x) -> np.ndarray:
    x = np.asarray(x) / np.pi
    x = np.abs(x)
    r = np.modf(x)
    r = r[0], r[1] % 2
    return np.abs(r[0] - r[1])

def sample_exact_thetas(v, *, n=1, seed=None):
    if isinstance(v, dict):
        dkeys = v.keys()
        v = np.array(list(v.values()))
    v = thetas_to_prob(v)
    rng = np.random.default_rng(seed)
    prob = rng.uniform(size=(n, len(v)))
    v = (prob < v) * np.pi
    if dkeys is not None:
        v = [dict(zip(dkeys, v1)) for v1 in v]
        assert len(v) == n
    return v

In [21]:
def RandomPermutationMatrix(n):
  p = np.arange(n) + 1
  np.random.shuffle(p)
  m = []
  for i in range(n):
    line = np.zeros(n).astype(int)
    line[p[i] - 1] = 1
    m.append(line)
  return np.array(m)

In [22]:
def brute_force_maximum_independent_set(adj_matrix):
    n = len(adj_matrix)
    max_set = []
    for i in range(2 ** n):
        binary_i = bin(i)[2:].zfill(n)
        curr_set = [j for j in range(n) if binary_i[j] == '1']
        is_independent = True
        for u in curr_set:
            for v in curr_set:
                if u != v and adj_matrix[u][v] == 1:
                    is_independent = False
                    break
            if not is_independent:
                break
        if is_independent and len(curr_set) > len(max_set):
            max_set = curr_set
    return max_set


In [23]:
def RandomGraph(n):
   g = nx.Graph()
   for i in range(n):
      for j in range(i+1, n):
         if np.random.randint(4) <= 0:
            g.add_edge(i, j)

   a = []
   for i in range(n):
      a.append(i)

   adj_g = nx.adjacency_matrix(g, a).todense()
   nx_ans = brute_force_maximum_independent_set(adj_g)
   return adj_g, nx_ans

In [24]:
def calc_adj_ans(adj: np.ndarray, n: int):
    for i in range(n):
        for j in range(n):
            if adj[i][j] == 1:
                return 0
    ans = 0
    for i in range(n):
        if adj[i][i] == -1:
            ans+=1
    return ans

def calc_adj_appro_ans(adj: np.ndarray, n: int):
    res = [0]
    for i in range(1, n):
        flag = True
        for el in res:
            if adj[el][i] != 0:
                flag = False
                break
        if flag:
            res.append(i)
    ans = 0
    for i in res:
        if adj[i][i] == -1:
            ans+=1
    return ans


In [25]:
def solveClique(n, adj, get_info):
    for i in range(n):
        adj[i][i]=-1
    
    adj = np.append(adj, np.zeros(n * n).reshape(n, n), axis=1)
    adj = np.append(adj, np.zeros(n * 2*n).reshape(n, 2*n), axis=0)
    adj += 1
    adj = adj.astype(int)

    a = np.zeros(2*n * 2*n)
    for i in range(n):
        for j in range(n):
            a[i * (2*n) + j] = 1

    n <<= 1

    P = RandomPermutationMatrix(n)
    adj = P @ adj @ P.T

    qbit = int(np.log2(n))
    qc, params = s4_ansatz('circular', qreg=qbit)
    ansatz = QuantumCircuit(qbit * 2)
    ansatz.compose(qc, inplace=True, qubits=range(qbit))
    ansatz.compose(qc, inplace=True, qubits=np.array(range(qbit)) + qbit)

    f = Framework()
    f.encode(adj.flatten())
    f.operation(a)
    f.set_ansatz(ansatz)

    qubits_number = 0
    depth = 0
    if get_info:
        qc = f.loss_qc()
        qc = transpile(qc, basis_gates=['cx', 'u1', 'u2', 'u3'])
        qubits_number = len(qc.qubits)
        depth = qc.depth()

    initial_point = (rng.uniform(size=len(qc.parameters)) - 1/2) * np.pi
    result = f.Run(seed=seed, init_point=initial_point)

    qc1 = s4_ansatz('circular', qreg=int(np.log2(len(adj))), params=params)[0]
    sampled_params_dicts = sample_exact_thetas(result.optimal_parameters,
                                                n=32, seed=seed)
    ret = 0
    appro_ret = 0
    p2 = []
    for v in sampled_params_dicts:
        p1 = np.abs(Operator(qc1.bind_parameters(v)).data)
        p1 = np.round(p1)
        ans = calc_adj_ans(p1 @ (adj - 1) @ p1.T, n>>1)
        appro_ans = calc_adj_appro_ans(p1 @ (adj - 1) @ p1.T, n>>1)
        if appro_ret < appro_ans:
            appro_ret = appro_ans
        if ret < ans:
            ret = ans
            p2 = p1 @ P
    
    return ret, appro_ret, p2, (qubits_number, depth)

In [26]:
def RandomCompleteGraphTest(n, RandomPermu=100):
    adj_g, nx_ans = RandomGraph(n)

    all_cost = []
    ret_info = ()
    for i in tqdm(range(RandomPermu)):
        P = RandomPermutationMatrix(n)
        adj = P @ adj_g @ P.T
        cost, appro_ans, p, info = solveClique(n, adj, i==0)
        if i == 0:
            ret_info = info
        all_cost.append([cost, appro_ans, p, P])

    return nx_ans, all_cost, adj_g, ret_info


In [27]:
ans, all_cost, adj_g, info = RandomCompleteGraphTest(8, 100)

quantum_ans = np.array(all_cost)[:, 0]
quantum_appro_ans = np.array(all_cost)[:, 1]

print(adj_g)
print("info: ", info)
print("answer: ", len(ans))
print("min: ", np.min(quantum_ans))
print("max: ", np.max(quantum_ans))
print("mean: ", np.mean(quantum_ans))
print("quantum_ans: ", quantum_ans)
print("quantum_appro_ans: ", quantum_appro_ans)

with open('./MIS/8-random-test100-1', 'a+') as f:
  print("seed= ", seed, file=f)
  print(adj_g, file=f)
  print("info: ", info, file=f)
  print("answer: ", len(ans), file=f)
  print("min: ", np.min(quantum_ans), file=f)
  print("max: ", np.max(quantum_ans), file=f)
  print("mean: ", np.mean(quantum_ans), file=f)
  print("quantum_ans: ", quantum_ans, file=f)
  print("quantum_appro_ans: ", quantum_appro_ans, file=f)
  print(all_cost, file=f)

  0%|          | 0/100 [00:00<?, ?it/s]/opt/homebrew/lib/python3.9/site-packages/numpy/linalg/linalg.py:2158: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.9/site-packages/numpy/linalg/linalg.py:2158: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
100%|██████████| 100/100 [2:09:24<00:00, 77.65s/it] 

[[0 0 1 0 0 0 0 0]
 [0 0 0 1 1 0 0 0]
 [1 0 0 0 0 0 1 1]
 [0 1 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 1 1 0 1 0 0]
 [0 0 1 0 0 0 0 0]]
info:  (10, 33179)
answer:  5
min:  2
max:  5
mean:  3.8
quantum_ans:  [3 4 4 4 4 4 5 4 3 5 4 4 4 4 5 4 3 3 4 4 4 3 4 4 5 4 4 3 4 4 2 5 4 4 4 3 3
 5 4 3 3 4 4 4 3 4 3 4 3 3 5 4 4 4 4 4 3 4 4 4 3 4 3 4 4 3 4 3 3 4 3 4 4 4
 5 3 4 3 4 5 4 3 4 4 4 3 4 5 4 4 4 4 2 4 4 4 4 4 3 4]
quantum_appro_ans:  [4 4 4 4 4 4 5 4 5 5 4 4 4 4 5 4 4 4 4 4 4 4 4 4 5 4 4 4 4 4 5 5 4 4 4 4 5
 5 4 4 5 4 4 4 4 4 4 4 4 4 5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 4 4 4 4
 5 4 4 4 4 5 4 4 4 4 4 4 4 5 4 4 4 5 5 4 4 4 5 4 5 4]



/var/folders/3b/54p896jd5hz9r642m7ngjx7h0000gn/T/ipykernel_60817/398529816.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  quantum_ans = np.array(all_cost)[:, 0]
/var/folders/3b/54p896jd5hz9r642m7ngjx7h0000gn/T/ipykernel_60817/398529816.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  quantum_appro_ans = np.array(all_cost)[:, 1]
